In [1]:
import requests
import bs4 as bs
import pandas as pd

#### First, we get our movie dataframe from the dataset given

In [2]:
columns = ["WikiID", "FreebaseID", "Name", "ReleaseDate", "BoxOfficeRevenue", "Runtime", "Language", "Country", "Genres"]
movies = pd.read_csv('../../data/movie.metadata.tsv', sep='\t', names=columns, index_col=False)

<h4> As we want to relate the FreebaseID we have with the IMDB id, we are going to look for the latter in <a> https://www.wikidata.org </a>. First, we get the path to the film web page with the function <i>getLinks</i>, and then we get the IMDB id with the function <i>getFreeIDandIMDBid</i>. Both functions are based on the HTML tags of said webpage after consultating it and makes requests to WikiData.

In [3]:
def getLinks (t):
    """
    Get the path to the film webpage
    Argument:
        t: it is a string that contains the name of the film already prepared for the URL.
        -----> t = "City+of+the+dead"
    Returns:
        An array with all the paths that wikidata has for the film
    """
    # This is the URL we need to search for the film webpage path
    addr = "https://www.wikidata.org/w/index.php?go=Go&search="+t+"&search="+t+"&title=Special:Search&ns0=1&ns120=1&searchToken=dattnx8r0gvcfzgyehr2g6vg2"
    
    # We request the previous URL and get the HTML
    r = requests.get(addr)
    soup = bs.BeautifulSoup(r.text, "html.parser")

    # This is the array that will contain all the paths and we are returning
    link = []

    # We know the path we are looking for is contained in a div with the class ="mw-search-result-heading", inside the element <a> with the tag "href"
    for div in soup.find_all('div',{"class":"mw-search-result-heading"}):
        for a in div:
            link.append(a.get("href"))
            break

    return link

In [4]:
def getFreeIDandIMDBid (link):
    """
    Get the Freebase id and the IMDB id of the film from its wikidata webpage
    Argument:
        link: it is a string with the path for the URL film webpage.
        -----> link = "wiki/Q869644"
    Returns:
        The Freebase id and the IMDB id of the film 
    """
    # We create the whole URL with the baseURL (addr2) and the path (link)
    addr2 = "https://www.wikidata.org"
    url = addr2 + link

    # We request the previous URL and get the HTML
    r2 = requests.get(url)
    soup2 = bs.BeautifulSoup(r2.text, "html.parser")

    # The variables we are returning, which will contain the Freebase id and IMDB id
    freebaseid = ""
    imbdid = ""

    # We know the Freebase id is contain in a div with id ="P646", in the element a with the class ="wb-external-id external"
    for div in soup2.find_all('div', {"id": "P646"}):
        for a in div.find_all('a', {"class": "wb-external-id external"}):
            freebaseid = a.get_text()
            break

    # We know the IMDB id is contain in a div with id ="P345", in the element a with the class ="wb-external-id external"
    for div in soup2.find_all('div', {"id": "P345"}):
        for a in div.find_all('a', {"class": "wb-external-id external"}):
            imbdid = a.get_text()
            break
    
    return freebaseid, imbdid

In [5]:
# First, we divide the number of movies into 5 (number of files to parallel the search)
top = round(len(movies)/5)

# We get an array with the names of the movies we are going to make the search for
## If it has crashed, it would be movies1 = movies[top*4+len(dict_fbid_to_imbd5):]
movies1 = movies[top*4:].Name


In [9]:
# This dict will have the Freebase id as key and the IMDB id as its value
dict_fbid_to_imbd5 = {}

# For every title movie in the array:
for m1 in movies1:
    # We prepare the title for introducing it into an URL
    t1 = ("+").join(m1.split(" "))
    # We get the paths for the movie webpage
    links1 = getLinks(t1)
    # The Freebase id and IMDB id we are going to store in the dict
    freebaseid1 = ""
    imbid1 = ""
    # We search for the Freebase id and IMDB id in every path we received until we got them
    for link1 in links1:
        fid1, iid1 = getFreeIDandIMDBid(link1)
        if (fid1 != "" and freebaseid1 ==""):
            freebaseid1 = fid1
        if (iid1 != "" and imbid1 ==""):
            imbid1 = iid1
        if (imbid1 != "" or freebaseid1 != ""):
            break
    dict_fbid_to_imbd5[freebaseid1] = imbid1

dict_fbid_to_imbd5



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


{'/m/057fyxq': 'tt0055960',
 '/m/0ndwt2w': 'tt0903624',
 '/m/0fpgpcq': 'tt0131027',
 '/m/08kbqg': 'tt0034342',
 '/m/0j24f6y': 'tt0083119',
 '/m/095zlp': 'tt0274558',
 '/m/012_t5': '',
 '/m/02pw3h0': 'tt0083349',
 '/m/0ch5xnf': 'tt0030436',
 '/m/01hbmbw': '',
 '/m/0bbvsgt': 'tt0259322',
 '/m/01_c4': '',
 '/m/011x3wnk': 'tt3900614',
 '/m/0crcvr0': 'tt0151370',
 '/m/0c8r6s': 'tt0770214',
 '/m/04n6gjz': 'tt0413072',
 '/m/04jmm74': 'tt0229539',
 '/m/027td8k': '',
 '/m/03t397': 'tt0070300',
 '/m/04dr76': 'tt0107421',
 '/m/09gkdck': 'tt0074634',
 '/m/022fb': '',
 '/m/0gx2khm': 'tt0176869',
 '/m/0hrh9yp': 'tt0408847',
 '/m/0h23m': '',
 '/m/06w6swv': 'tt0387735',
 '': '',
 '/m/010hpqjd': 'tt0021031',
 '/m/0r4vvxq': '',
 '/m/04jf1pd': '',
 '/m/0dygx8': 'tt0375066',
 '/m/05c34sj': 'tt0453486',
 '/m/0bj4ft3': 'tt0027331',
 '/m/03c055l': 'tt0017855',
 '/m/0gfgsqy': 'tt8541250',
 '/m/0fyyrs8': '',
 '/m/0kvg4h': 'tt0246315',
 '/m/0bbv0qf': 'tt0049286',
 '/m/0gg6jtf': 'tt0183483',
 '/m/0j45hgz': 'tt02

In [10]:
# We store the dict as a global variable, so we can use it in another file
%store dict_fbid_to_imbd5

Stored 'dict_fbid_to_imbd5' (dict)
